In [2]:
from shapely import from_wkt
import networkx as nx
import osmnx as ox
from src.dongraphio.utils.graphs import get_osmnx_graph

G_drive: nx.MultiDiGraph = get_osmnx_graph(
    176095, 32636, "drive"
)
nx.write_graphml(G_drive, "len_obl_graph.graphml")
for i in G_drive.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
gdf = ox.graph_to_gdfs(G_drive, nodes=False)
gdf.to_file("len_obl_graph.geojson")

2024-03-13 15:07:03.922 | DEBUG    | src.dongraphio.utils.graphs:get_osmnx_graph:54 - Extracting and preparing drive graph from OSM ...
C:\Users\Donny\AppData\Local\pypoetry\Cache\virtualenvs\graph-lib-8TUFT-q0-py3.11\Lib\site-packages\osmnx\_overpass.py:211: UserWarning: This area is 51 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [5]:
G_drive_c = G_drive.copy()
for i in G_drive_c.edges(data=True):
    i[2]['geometry'] = (str(i[2]['geometry']))
nx.write_graphml(G_drive_c, "len_obl_graph.graphml")

In [1]:
import geopandas as gpd

stops_gdf = gpd.read_file("stopsLO.geojson")
stops_gdf.rename(columns={"Автоб": "route"}, inplace=True)
stops_gdf.dropna(subset="route", axis=0, inplace=True, how="any")
stops_gdf


,name,route,geometry
0,Дусьево,"593,593/596,847,856,860,860Л,864,865,865Д,867,...",POINT (31.72832 59.93558)
1,None,247,POINT (32.44568 59.28601)
2,Горная Шальдиха,"588,589,590,593,593/594",POINT (31.46034 59.87465)
3,Мучихино,590,POINT (31.52410 59.86666)
4,Васильково,590,POINT (31.58237 59.87627)
...,...,...,...
8386,\nЗаостровье,94,POINT (33.27731 60.62779)
8387,\nЗаостровье,94,POINT (33.27741 60.62765)
8388,Горка,94,POINT (33.22445 60.63948)
8389,Новая Слобода,"89А,91",POINT (33.33512 60.78394)


In [2]:
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()
unique_routes = set()
for index, row in (stops_gdf.iterrows()):
    for i in str(row["route"]).replace('.', ',').rstrip().lstrip().split(","):
        if i != "":
            unique_routes.add(i.lstrip().rstrip())
unique_routes = pd.DataFrame(index=list(unique_routes))
unique_routes["geometry"] = [[]] * len(unique_routes)
for index, row in (stops_gdf.iterrows()):
    for i in str(row["route"]).replace('.', ',').rstrip().lstrip().split(","):
        if i != "":
            unique_routes.loc[i.lstrip().rstrip(), "geometry"] = unique_routes.loc[i.lstrip().rstrip()]["geometry"] + [
                row["geometry"]]

unique_routes

,geometry
596А,"[POINT (30.5129796 59.903835), POINT (30.51203..."
79/77,"[POINT (28.7021229 59.4006375), POINT (28.7004..."
205 Красный,"[POINT (30.461842 60.0473808), POINT (30.46284..."
151а/151,"[POINT (33.5536502 59.6433943), POINT (33.5772..."
76Б,"[POINT (28.7021229 59.4006375), POINT (28.7004..."
...,...
348,"[POINT (29.93926291249601 59.834758554893504),..."
32Б,"[POINT (29.5023117 59.4695373), POINT (29.5017..."
12а,"[POINT (33.515119 59.6454752), POINT (33.49522..."
481,"[POINT (30.1444949 59.7893016), POINT (30.1432..."


In [3]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
from shapely import Point


def clusterize(loc) -> pd.DataFrame:
    data = np.array([[p.x, p.y] for p in loc["geometry"]])
    noise = np.array([31.212390, 58.611219])
    data = np.append(data, [noise], axis=0)

    best_silhouette = -1
    best_labels = None
    k_values = range(2, 10 if len(data) > 10 else 2)

    for k in k_values:
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(data)
        labels = kmeans.labels_
        silhouette = silhouette_score(data, labels)
        if silhouette > best_silhouette:
            best_silhouette = silhouette
            best_labels = labels

    data = data[:-1]
    if best_labels is not None:
        best_labels = best_labels[:-1]

    if best_silhouette < 0.93:
        best_labels = [0 for _ in range(0, len(data))]
    data = pd.DataFrame({
        'label': best_labels,
        'geometry': [Point(p[0], p[1]) for p in data]
    })
    grouped_data = data.groupby('label')['geometry'].apply(list)
    grouped_data = pd.DataFrame(grouped_data)
    grouped_data.index = grouped_data.index + 1
    return grouped_data



In [4]:
devided_routes = pd.DataFrame()
basic_routes = pd.DataFrame()
for index, row in tqdm(unique_routes.iterrows(), total=unique_routes.shape[0]):
    clusterized = clusterize(row)
    if clusterized.shape[0] > 1:
        clusterized.index = index + "__" + clusterized.index.astype(str)
        devided_routes = pd.concat([devided_routes, clusterized])
    else:
        clusterized.index = [index]
        basic_routes = pd.concat([basic_routes, clusterized])
devided_routes

  0%|          | 0/593 [00:00<?, ?it/s]

,geometry
label,
9__1,"[POINT (32.0151585 59.4556927), POINT (32.0151..."
9__2,"[POINT (29.0938556 59.9105085), POINT (29.0855..."
6__1,"[POINT (30.68372754798385 59.99380010324972), ..."
6__2,"[POINT (34.2174528 60.9123698), POINT (34.2251..."
6__3,"[POINT (28.6217408 59.3642358), POINT (28.6330..."
...,...
107__2,"[POINT (28.098742562071617 59.12618473418365),..."
95__1,"[POINT (28.0530789 59.0627202), POINT (28.0547..."
95__2,"[POINT (33.5826273 60.7344281), POINT (33.5820..."


In [5]:
row = devided_routes.loc["1__1"]
geometry_list = row["geometry"]
test_gdf = gpd.GeoDataFrame(data={"label":["1__1" for _ in geometry_list],"geometry":geometry_list})
test_gdf

,label,geometry
0,1__1,POINT (29.84695 58.71778)
1,1__1,POINT (29.84646 58.72545)
2,1__1,POINT (29.84644 58.72463)
3,1__1,POINT (29.84296 58.72715)
4,1__1,POINT (29.84783 58.72845)
5,1__1,POINT (29.84985 58.73366)
6,1__1,POINT (29.84944 58.73685)
7,1__1,POINT (29.85186 58.74097)
8,1__1,POINT (29.85380 58.74378)
9,1__1,POINT (29.85720 58.74918)


In [6]:
import networkx as nx
nx_graph = nx.read_graphml("len_obl_graph.graphml")

In [7]:
from src.dongraphio import DonGraphio
from src.dongraphio import GraphType
test_gdf = test_gdf.set_crs(4326,allow_override=True)
test_ = test_gdf.to_crs(32636)
dngrp = DonGraphio(city_crs=32636)
dngrp.set_graph(nx_graph)
adj_matrix = dngrp.get_adjacency_matrix(test_,test_,weight="length_meter",graph_type={GraphType.DRIVE})
adj_matrix

2024-03-13 16:16:15.495 | INFO     | src.dongraphio.dongraphio:get_adjacency_matrix:87 - Creating adjacency matrix based on provided graph...
2024-03-13 16:16:33.175 | DEBUG    | src.dongraphio.base_models:get_adjacency_matrix:93 - Calculating distances from buildings to services ...
2024-03-13 16:16:33.675 | INFO     | src.dongraphio.dongraphio:get_adjacency_matrix:96 - Adjacency matrix done!


,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.0,911.3,911.3,1422.9,1382.1,1966.3,2330.3,2978.5,3174.9,3813.2,...,4616.9,3071.4,0.0,3133.9,4837.8,6851.3,6851.3,537.5,537.5,766.1
1,911.3,0.0,0.0,511.6,470.8,1055.0,1419.0,2067.2,2263.6,2901.9,...,3705.6,2160.1,911.3,2222.6,3926.5,5940.0,5940.0,1448.8,1448.8,1677.4
2,911.3,0.0,0.0,511.6,470.8,1055.0,1419.0,2067.2,2263.6,2901.9,...,3705.6,2160.1,911.3,2222.6,3926.5,5940.0,5940.0,1448.8,1448.8,1677.4
3,1422.9,511.6,511.6,0.0,513.6,1097.8,1461.8,2110.0,2306.4,2944.7,...,3748.4,2202.9,1422.9,2265.4,3969.3,5982.8,5982.8,1960.4,1960.4,2189.0
4,1924.7,1013.4,1013.4,501.8,0.0,584.2,948.2,1596.4,1792.8,2431.1,...,3234.8,1689.3,1924.7,1751.8,3455.7,5469.2,5469.2,2462.2,2462.2,2690.8
5,2552.8,1641.5,1641.5,1129.9,1167.3,0.0,364.0,1012.2,1208.6,1846.9,...,2650.6,1105.1,2552.8,1167.6,2871.5,4885.0,4885.0,3090.3,3090.3,3318.9
6,2677.4,1766.1,1766.1,1254.5,1291.9,691.6,0.0,648.2,844.6,1482.9,...,2286.6,741.1,2677.4,803.6,2507.5,4521.0,4521.0,3214.9,3214.9,3443.5
7,3338.1,2426.8,2426.8,1915.2,1952.6,1352.3,959.1,0.0,196.4,834.7,...,1638.4,490.2,3338.1,155.4,1859.3,3872.8,3872.8,3875.6,3875.6,4104.2
8,3762.1,2850.8,2850.8,2339.2,2376.6,1776.3,1383.1,424.0,0.0,638.3,...,1442.0,914.2,3762.1,579.4,1662.9,3676.4,3676.4,4299.6,4299.6,4528.2
9,4188.8,3277.5,3277.5,2765.9,2803.3,2203.0,1809.8,1161.5,1164.4,0.0,...,803.7,1340.9,4188.8,1006.1,1024.6,3038.1,3038.1,4726.3,4726.3,4954.9


In [ ]:
from shapely.geometry import MultiPoint

devided_routes["geometry"] = devided_routes["geometry"].apply(lambda x: MultiPoint(x))
devided_routes = gpd.GeoDataFrame(data=devided_routes, geometry="geometry")
devided_routes.to_file("new_routes.geojson")
